In [1]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
import os

def load_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    return hsv_img


def mask_color(img, lower_color, upper_color):
    mask = cv2.inRange(img, lower_color_mask, upper_color_mask)

    kernel_open = np.ones((10,10))
    kernel_close = np.ones((50,50))

    mask_open = cv2.morphologyEx(mask,cv2.MORPH_OPEN, kernel_open)
    mask_close = cv2.morphologyEx(mask_open,cv2.MORPH_CLOSE, kernel_close)
    dilation = cv2.dilate(mask_close, np.ones((20, 20)), iterations = 1)

    masked = cv2.bitwise_and(img, img, mask=dilation)
    return masked


def detect_bounds(img):
    [y, x, z] = img.shape 
    mins = [y, x] # y,x
    maxes = [0, 0]
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            if not img[y][x][0] == 0:
                if y < mins[0]:
                    mins[0] = y
                if y > maxes[0]:
                    maxes[0] = y
                if x < mins[1]:
                    mins[1] = x
                if x > maxes[1]:
                    maxes[1] = x
    mins.reverse()
    maxes.reverse()            
    return [(mins[0], maxes[1]), tuple(maxes), (maxes[0], mins[1]), tuple(mins)]


def bounds_from_file(path, lower_color_mask, upper_color_mask):
    img = load_img(path)
    masked = mask_color(img, lower_color_mask, upper_color_mask)
    return detect_bounds(masked)

In [3]:
unlabeled_dir_path = "/mnt/Data/Documents/Praca_magisterska/dataset/dataset-master/unlabeled"
lower_color_mask = np.array([120,70,180])
upper_color_mask = np.array([140,140,240])

for udir in os.listdir(unlabeled_dir_path):
    udir_path = os.path.join(unlabeled_dir_path, udir)
    if os.path.isdir(udir_path):
        files = os.listdir(udir_path)
        skipped = 0
        with open(os.path.join(unlabeled_dir_path, udir + ".csv"), "w") as labels:
            for file in files:
                file_path = os.path.join(udir_path, file)
                img = load_img(file_path)
                masked = mask_color(img, lower_color_mask, upper_color_mask)
                bounds = detect_bounds(masked)
                (y, x, z) = img.shape
                if (bounds[0] == (x, 0) and bounds[3] == (x, y)):
                    skipped += 1
                else:
                    labels.write("{},{},{},{},{},{},{},{},{}\n".format(file, bounds[0][0], bounds[0][1], bounds[1][0], bounds[1][1], bounds[2][0], bounds[2][1], bounds[3][0], bounds[3][1]))
        print("Skipped " + str(skipped))

Skipped 624
Skipped 555
Skipped 182
Skipped 9
